<a href="https://colab.research.google.com/github/bpratham2001/ResNet/blob/main/Mass_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mass Regression

Please train a model to estimate (regress) the mass of the particle based on particle images using the provided dataset.

DataSet Description: 125x125 image matrices with name of variables: ieta and iphi, with 4 channels called X_jet (Track pT, DZ and D0, ECAL). Please use at least ECAL and Track pT channels and ‘am‘ as the target feature. If there are more than 4 channels in the dataset then you should use X_jet (Track pT, DZ and D0, ECAL) only. Please train your model on 80% of the data and evaluate on the remaining 20%. Please make sure not to overfit on the test dataset - it will be checked with an independent sample.

Datasets: https://cernbox.cern.ch/s/zUvpkKhXIp0MJ0g

**1,051,917 rows in total**

In [1]:
# @title !wget .parquet files

#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_0.parquet
#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_1.parquet
#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_2.parquet
#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_3.parquet
#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_4.parquet
#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_5.parquet
#!wget https://cernbox.cern.ch/remote.php/dav/public-files/zUvpkKhXIp0MJ0g/top_gun_opendata_6.parquet

parqs = ['top_gun_opendata_0.parquet', 'top_gun_opendata_1.parquet', 'top_gun_opendata_2.parquet',
         'top_gun_opendata_3.parquet', 'top_gun_opendata_4.parquet', 'top_gun_opendata_5.parquet',
         'top_gun_opendata_6.parquet']

from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/My Drive/top_gun/'
#folder = '/kaggle/working/'

Mounted at /content/drive


In [2]:
# @title imports and installs

#import pyspark
#from pyspark.sql import SparkSession
#import pandas as pd
#from IPython.display import clear_output
import random
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import datetime
from math import sqrt
import gc
#from sklearn.metrics import root_mean_squared_error

#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
# @title Model Architecture (ResNet)

class ResBlock(nn.Module):
  # This is the residual block, there are 4 residual blocks in ResNet15
  def __init__(self, in_channels, out_channels, stride=1):
    super(ResBlock, self).__init__()
    self.relu = nn.ReLU(inplace=True)
    self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1, stride=stride, padding=0, bias=False)
    nn.init.kaiming_normal_(self.conv1.weight)
    self.bn1 = nn.BatchNorm2d(in_channels)

    self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=False)
    nn.init.kaiming_normal_(self.conv2.weight)
    self.bn2 = nn.BatchNorm2d(in_channels)

    self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
    nn.init.kaiming_normal_(self.conv3.weight)
    self.bn3 = nn.BatchNorm2d(out_channels)

    self.shortcut = nn.Sequential()

    if stride != 1 or in_channels != out_channels:
      shortcut_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, padding=0, bias=False)
      nn.init.kaiming_normal_(shortcut_conv.weight)
      self.shortcut.append(shortcut_conv)
      self.shortcut.append(nn.BatchNorm2d(out_channels))

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)
    out = self.conv3(out)
    out = self.bn3(out)
    out += self.shortcut(x)
    out = self.relu(out)
    return out

class ResNet(nn.Module):
  def __init__(self, num_classes=2):
    super(ResNet, self).__init__()
    self.conv1 = nn.Conv2d(4, 16, kernel_size=1, stride=1, padding=0, bias=False)
    nn.init.kaiming_normal_(self.conv1.weight)
    self.bn1 = nn.BatchNorm2d(16)
    self.relu = nn.ReLU(inplace=True)

    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

    self.res1 = ResBlock(16, 32)
    self.res2 = ResBlock(32, 64)

    self.conv2 = nn.Conv2d(64, 128, kernel_size=1, stride=1, padding=0, bias=False)
    nn.init.kaiming_normal_(self.conv2.weight)
    self.bn2 = nn.BatchNorm2d(128)

    self.res3 = ResBlock(128, 256)
    self.res4 = ResBlock(256, 512)

    self.fc1 = nn.Linear(512 * 125 * 125, 1)
    nn.init.kaiming_normal_(self.fc1.weight)
    #self.fc2 = nn.Linear(16 * 125 * 125, 1)
    #nn.init.kaiming_normal_(self.fc2.weight)

  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)
    out = self.maxpool(out)
    out = self.res1(out)
    out = self.res2(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = self.res3(out)
    out = self.res4(out)
    out = torch.flatten(out, 1)
    out = self.fc1(out)
    #out = self.fc2(out)
    out = self.relu(out)
    return out

In [4]:
# @title dataset metadata and helper functions

def get_parquet_rows(parq, start=0, end=-1, drive=False):
  # since rows == row_groups in the dataset, we can use them interchangeably
  lst = []
  if drive:
    parq_file = pq.ParquetFile(folder+parq)
  else:
    parq_file = pq.ParquetFile(parq)
  if end < 0:
    end = parq_file.scan_contents()
  while start < end:
    lst.append(parq_file.read_row_group(start))
    start += 1
  return lst

def make_X(X_jet):
  X_set = np.zeros(shape=(4, 125, 125))
  for i in range(4): # only selecting the first 4 channels (Track pT, DZ, D0, ECAL)
    for j in range(125):
      for k in range(125):
        X_set[i][j][k] = X_jet[i][j][k].astype(float)
  return X_set
  #return torch.tensor(X_set, dtype=torch.float32)

def make_Xs(X_jets, chunk):
  X_sets = np.zeros(shape=(chunk, 4, 125, 125))
  for i in range(chunk):
    for j in range(4):
      for k in range(125):
        for l in range(125):
          X_sets[i][j][k][l] = X_jets[i][j][k][l].astype(float)
  return X_sets


def get_Xy(parq, start=0, end=-1, drive=False):
  arr = np.array(get_parquet_rows(parq, start, end, drive))
  X = []
  y = []
  for i in range(len(arr)):
    X.append(make_X(np.array(arr[i][0][0])))
    y.append(arr[i][0][1])
  return np.array(X), np.array(y)

""" # row counter
tot = 0
for parq in parqs:
  num = pq.ParquetFile(folder+parq).scan_contents()
  tot += num
  print(parq+" contains "+str(num)+" rows")
print("total " + str(tot))
del tot, num

top_gun_opendata_0.parquet contains 150327 rows
top_gun_opendata_1.parquet contains 150165 rows
top_gun_opendata_2.parquet contains 150451 rows
top_gun_opendata_3.parquet contains 150448 rows
top_gun_opendata_4.parquet contains 150557 rows
top_gun_opendata_5.parquet contains 150056 rows
top_gun_opendata_6.parquet contains 149913 rows
total 1051917
"""

for parq in parqs:
  break
  print(pq.read_metadata(folder+parq))
  #print(pq.ParquetFile(folder+parq).read_row_group(0))

class Deck():
  def __init__(self, filenames, filesizes):
    self.filenames = filenames
    self.filesizes = filesizes
    self.deck = []

  def create_deck(self, max_chunk):
    self.deck = []
    for i in range(len(self.filenames)):
      start = 0
      while self.filesizes[i] > 0:
        if max_chunk >= self.filesizes[i]:
          self.deck.append([self.filenames[i], start, (start+self.filesizes[i])])
          self.filesizes[i] = 0
        else:
          self.deck.append([self.filenames[i], start, (start+max_chunk)])
          self.filesizes[i] -= max_chunk
          start += max_chunk

  def shuffle_deck(self):
    random.shuffle(self.deck)

  def deal_cards(self, split):
    return self.deck[:int(len(self.deck)*split)], self.deck[int(len(self.deck)*split):]

  def play_card(self, card, batch_size, drive=False, reshuffle=True):
    if drive:
      parq_file = pq.ParquetFile(folder+card[0])
    else:
      parq_file = pq.ParquetFile(card[0])

    X = parq_file.read_row_groups(np.arange(card[1], card[2]), columns=['X_jet', 'm']).to_pandas()
    torch.cuda.empty_cache()
    y = torch.tensor(X['m'].to_numpy(), dtype=torch.float32)
    X = make_Xs(X['X_jet'].to_numpy(), card[2]-card[1]) # Works, but slow
    #X_tensor = torch.from_numpy(df['X_jet'].to_numpy(dtype=float))
    X = torch.tensor(X, dtype=torch.float32)
    #print("X_tensor", str(X.shape))
    #print("y_tensor", str(y.shape))
    X = X.to(device)
    y = y.to(device)
    return DataLoader(TensorDataset(X, y), batch_size=batch_size, shuffle=reshuffle)

In [5]:
"""
1/1000 : 029 seconds
1/4000 : 125 seconds
2/4000 : 120 seconds
4/4000 :
8/4000 :
"""
import multiprocessing
def make_tensors(start):
  parq_file = pq.ParquetFile(folder+parqs[1])
  X = parq_file.read_row_groups(np.arange(start, start+1000), columns=['X_jet', 'm']).to_pandas()
  y = torch.tensor(X['m'].to_numpy(), dtype=torch.float32)
  X = torch.tensor(X['X_jet'].to_list(), dtype=torch.float32)
  return X[:,:4], y

print(datetime.datetime.now())
"""
s = [1000, 2000]
parq_file = pq.ParquetFile(folder+parqs[1])
X = parq_file.read_row_groups(np.arange(s[0], s[1]), columns=['X_jet', 'm']).to_pandas()
y = torch.tensor(X['m'].to_numpy(), dtype=torch.float32)
#X = make_Xs(X['X_jet'].to_numpy(), s[1]-s[0]) # Works, but slow
"""

if __name__ == '__main__':
  pool = multiprocessing.Pool()
  pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
  #X = np.array(pool.map(make_X2, X['X_jet'].to_numpy()))
  X = pool.map(make_tensors, [0, 1066, 2016, 3033])

y = torch.cat([X[i][1] for i in range(4)],0)
X = torch.cat([X[i][0] for i in range(4)],0)

##outputs_async = pool.map_async(make_X, X['X_jet'].to_numpy())
##outputs = outputs_async.get()

print(X.shape)
print(y.shape)
print(datetime.datetime.now())
del X, y
gc.collect()

2025-04-09 21:51:31.949105


<ipython-input-5-56d55c91d4c2>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X = torch.tensor(X['X_jet'].to_list(), dtype=torch.float32)
<ipython-input-5-56d55c91d4c2>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X = torch.tensor(X['X_jet'].to_list(), dtype=torch.float32)


torch.Size([4000, 4, 125, 125])
torch.Size([4000])
2025-04-09 21:53:33.707055


30

In [ ]:
from multiprocessing import Pool
print(datetime.datetime.now())
if __name__ == '__main__':
  with Pool(4) as p:
    X = p.map(make_tensors, [0, 1066, 2016, 3033])
    p.close()
    p.join()
y = torch.cat([X[i][1] for i in range(4)],0)
X = torch.cat([X[i][0] for i in range(4)],0)
print(X.shape)
print(y.shape)
#X = torch.tensor(make_Xs(df['X_jet'].values, 4000), dtype=torch.float32) #10min
print(datetime.datetime.now())
del X, y
gc.collect()
#!pip uninstall -y tensorflow
#!pip install tensorflow-cpu

2025-04-09 21:59:53.334982


<ipython-input-5-56d55c91d4c2>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X = torch.tensor(X['X_jet'].to_list(), dtype=torch.float32)
<ipython-input-5-56d55c91d4c2>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  X = torch.tensor(X['X_jet'].to_list(), dtype=torch.float32)
Process ForkPoolWorker-8:
Process ForkPoolWorker-11:
Process ForkPoolWorker-7:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.11/mult

In [ ]:
# @title Tuning and Loading the Model

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
#import torch_xla.distributed.parallel_loader as pl
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
###############################################################################
epochs = 2
split = 0.8
max_chunk = 2500
batch_size = 250
optimiseur = ['Adam', 0.001] # ['Adam, learning rate'] / ['SGD', learning rate, momentum] // scaled_eta = eta * xm.xrt_world_size()
criterion = nn.MSELoss()#.to(device)
path = '' # 'mass_regressor.pth'
###############################################################################

model = ResNet()
mx = xmp.MpModelWrapper(model)

if torch.cuda.device_count() > 1:
  print("Using ", torch.cuda.device_count(), "GPUs")
  model = nn.DataParallel(model)

if path != '':
  model.load_state_dict(torch.load(f=path))

In [ ]:
# @title averageMeter
class AverageMeter(object):

    '''Computes and stores the average and current value'''

    def __init__(self):
        self.reset()

    def reset(self):
        self.val   = 0
        self.avg   = 0
        self.sum   = 0
        self.count = 0

    def update(self, val, n = 1):
        self.val    = val
        self.sum   += val * n
        self.count += n
        self.avg    = self.sum / self.count

In [ ]:
#model = model.to(device)

if optimiseur[0] == 'Adam':
  optimiseur = torch.optim.Adam(model.parameters(),lr=optimiseur[1])
elif optimiseur[0] == 'SGD':
  optimiseur = torch.optim.SGD(model.parameters(), lr=optimiseur[1], momentum=optimiseur[2])

param_size = 0
for param in model.parameters():
  param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
  buffer_size += buffer.nelement() * buffer.element_size()

print('model size: {:.3f}MB'.format((param_size + buffer_size) / 1024**2))

model size: 35.034MB


In [ ]:
# @title Training

# para_loader = pl.ParallelLoader(valid_loader, [device])
# para_loader = para_loader.per_device_loader(device)

def train(model):
  print("Training started at", datetime.datetime.now())

  # send to TPU
  device = xm.xla_device()
  model  = mx.to(device)

  parq_len = [150327, 150165, 150451, 150448, 150557, 150056, 149913]
  deck = Deck(parqs, parq_len)
  deck.create_deck(max_chunk)
  deck.shuffle_deck()
  train_cards, test_cards = deck.deal_cards(split)
  plotloss = []
  flag = False
  model.train()
  trn_loss_meter = AverageMeter()
  scheduler = torch.optim.lr_scheduler.StepLR(optimiseur, step_size = 1, gamma = 0.5)

  gc.collect()
  for epoch in range(epochs):
    """
    if (epochs > 5) and (epoch > (epochs*0.5)):
      optimiseur = torch.optim.Adam(model.parameters(),lr=0.001)
    elif (epochs > 5) and (epoch > (epochs*0.75)):
      optimiseur = torch.optim.Adam(model.parameters(),lr=0.0001)
    """
    #running_loss = 0.0 # MSE, per epoch

    for card in train_cards:
      #acc = [] # RMSE, per card per epoch
      #print(card)
      for i, (inputs, labels) in enumerate(deck.play_card(card, batch_size, drive=True, reshuffle=False)):
        #print('{}MB'.format(inputs.nelement() * inputs.element_size() / 1024**2))
        #print('{}MB'.format(labels.nelement() * labels.element_size() / 1024**2))
        outputs = model(inputs)
        if flag:
          print(outputs) #torch.flatten(torch.rot90(outputs)))
          print(labels) #torch.flatten(labels))
          flag = False
        loss = criterion(outputs, labels.unsqueeze(1))
        optimiseur.zero_grad()
        loss.backward()
        #optimiseur.step()
        xm.optimizer_step(optimiseur, barrier=True) # barrier is required on single-core training but can be dropped with multiple cores
        #running_loss += loss.item()
        trn_loss_meter.update(loss.detach().item(), inputs.size(0))
        #acc.append(sqrt(loss))
      #print(f"RMSE: {sum(acc)/len(acc)}")
      xm.master_print('E{}, {} | loss = {:.6f}, avg_loss = {:.6f}'.format(epoch, card, loss.item(), trn_loss_meter.avg))

      del inputs, labels, outputs, loss
      gc.collect()
      scheduler.step()
    plotloss.append(trn_loss_meter.avg)
    #print(f"E [{epoch + 1}/{epochs}], L: {running_loss:.4f}, T: [{datetime.datetime.now()}]")

  print("Training completed at", datetime.datetime.now())
  return plotloss

# wrapper function
def _mp_fn(rank, flags):
  torch.set_default_tensor_type('torch.FloatTensor')
  trn_losses = train(model)
  np.save('trn_losses.npy', np.array(trn_losses))
  #np.save('val_losses.npy', np.array(val_losses))

# modeling
gc.collect()
FLAGS = {}
xmp.spawn(_mp_fn, args = (FLAGS,), nprocs = None, start_method = 'fork')

In [ ]:
import seaborn as sns
sns.lineplot(x=np.array(list(range(epochs))), y=np.array(plotloss))

In [ ]:
# @title Testing

print("Testing started at", datetime.datetime.now())

val_loss = 0.0
acc = []
model.eval()
with torch.no_grad():
  for card in test_cards:
    for i, (inputs, labels) in enumerate(deck.play_card(card, batch_size, drive=True, reshuffle=False)):
      outputs = model(inputs)
      loss = criterion(outputs, labels.unsqueeze(1))
      val_loss += loss.item()
      acc.append(sqrt(loss))
    print(sqrt(loss))

print(f"avg RMSE: {sum(acc)/len(acc)}")
print("Testing completed at", datetime.datetime.now())

In [ ]:
# @title Saving
torch.save(obj=model.state_dict(), f="resnet_regressor.pth")

# Tensorflow stuffs to ignore

In [ ]:
# @title imports and installs
!pip install tensorflow==2.18.0
!pip install tensorflow-tpu==2.18.0 --find-links=https://storage.googleapis.com/libtpu-tf-releases/index.html
#import pyspark
#from pyspark.sql import SparkSession
#import pandas as pd
from IPython.display import clear_output
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import tensorflow as tf
import tensorflow.keras as keras
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
#import tensorflow_datasets as tfds
import datetime
from math import sqrt
#from sklearn.metrics import root_mean_squared_error

from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# @title Tuning
#TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

#optimiseur = keras.optimizers.SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04)
optimiseur = keras.optimizers.Adam(learning_rate=0.01)
criterion = keras.losses.MeanSquaredError()
batch_size_parq = 5000
batch_size_train = 256
steps_per_epoch = 50000 // batch_size_train
exec_step = 50 # between 2 and steps_per_epoch
epochs = 10

In [ ]:
# @title dataset metadata and helper functions

def get_parquet_rows(parq, start=0, end=-1, drive=False):
  # since rows == row_groups in the dataset, we can use them interchangeably
  lst = []
  if drive:
    parq_file = pq.ParquetFile(folder+parq)
  else:
    parq_file = pq.ParquetFile(parq)
  if end < 0:
    end = parq_file.scan_contents()
  while start < end:
    lst.append(parq_file.read_row_group(start))
    start += 1
  return lst

def make_X(X_jet):
  X_set = np.zeros(shape=(4, 125, 125))
  for i in range(4): # only selecting the first 4 channels (Track pT, DZ, D0, ECAL)
    for j in range(125):
      for k in range(125):
        X_set[i][j][k] = X_jet[i][j][k]
  return tf.convert_to_tensor(X_set, tf.float32)

def make_Xs(X_jets):
  X_sets = np.zeros(shape=(len(X_jets), 4, 125, 125))
  for i in range(len(X_jets)):
    for j in range(4): # only selecting the first 4 channels (Track pT, DZ, D0, ECAL)
      for k in range(125):
        for l in range(125):
          X_sets[i][j][k][l] = X_jets[i][j][k][l]
  return tf.convert_to_tensor(X_sets, tf.float32)


def get_Xy(parq, start=0, end=-1, drive=False):
  arr = np.array(get_parquet_rows(parq, start, end, drive))
  X = []
  y = []
  for i in range(len(arr)):
    X.append(make_X(np.array(arr[i][0][0])))
    y.append(arr[i][0][1])
  return np.array(X), np.array(y)

""" # row counter
tot = 0
for parq in parqs:
  num = pq.ParquetFile(folder+parq).scan_contents()
  tot += num
  print(parq+" contains "+str(num)+" rows")
print("total " + str(tot))
del tot, num

top_gun_opendata_0.parquet contains 150327 rows
top_gun_opendata_1.parquet contains 150165 rows
top_gun_opendata_2.parquet contains 150451 rows
top_gun_opendata_3.parquet contains 150448 rows
top_gun_opendata_4.parquet contains 150557 rows
top_gun_opendata_5.parquet contains 150056 rows
top_gun_opendata_6.parquet contains 149913 rows
total 1051917
"""
parq_len = [150327, 150165, 150451, 150448, 150557, 150056, 149913]

for parq in parqs:
  print(pq.read_metadata(folder+parq))
  #print(pq.ParquetFile(folder+parq).read_row_group(0))
  break

"""
print(datetime.datetime.now())
parq_file = pq.ParquetFile(folder+parqs[0])
for i in parq_file.iter_batches(batch_size=4000, columns=['X_jet', 'm']):
  df = i.to_pandas()
  df['X_jet'] = df['X_jet'].map(lambda x: make_X(x))
  break
print(df['X_jet'].iloc[0].shape)
print(datetime.datetime.now())
"""

In [ ]:
# @title tf_recordwriter

def read_tfrecord(example):
  features={
    "X": tf.io.FixedLenFeature([], tf.string),
    "y": tf.io.FixedLenFeature([], tf.float32),
    }
  example = tf.io.parse_single_example(example, features)
  #X = make_X(example["X_track_pT", "X_DZ", "X_D0", "X_ECAL"])
  X = tf.io.parse_tensor(example['X'], tf.float32)
  y = tf.cast(example['y'], tf.float32)
  return X, y

def get_datasets(tfRecords, batch_size, shuffle=5000, multiple=False):
  #For optimal performance, read multiple TFRecord files and set option experimental_deterministic = False
  dataset = tf.data.TFRecordDataset(tfRecords, num_parallel_reads=tf.data.experimental.AUTOTUNE)
  option = tf.data.Options()
  if multiple:
    option.experimental_deterministic = False
  dataset = dataset.with_options(option)
  dataset = dataset.map(read_tfrecord)
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  dataset = dataset.shuffle(shuffle)
  dataset = dataset.batch(batch_size)
  return dataset

with tf.io.TFRecordWriter("top_gun.tfrecords") as file_writer:
  print("Writing started at", datetime.datetime.now())
  for i in range(len(parqs)):
    for j in range(parq_len[i]):
      X, y = get_Xy(parqs[i], start=j, end=(j+4000), drive=True)
      X_serialized = tf.io.serialize_tensor(X)
      j += 5000
      features = {
          "X": tf.train.Feature(bytes_list=tf.train.BytesList(value=[X_serialized.numpy()])),
          "y": tf.train.Feature(float_list=tf.train.FloatList(value=y)),
          }
      features = tf.train.Features(feature=features)
      example = tf.train.Example(features=features)
      record_bytes = example.SerializeToString()
      file_writer.write(record_bytes)
      print(datetime.datetime.now())
    print(f"parq {parq[i]} complete")
  print("Writing completed at", datetime.datetime.now())

In [ ]:
# @title ResNet18 Architecture
"""
ResNet-18
Reference:
[1] K. He et al. Deep Residual Learning for Image Recognition. CVPR, 2016
[2] K. He, X. Zhang, S. Ren, and J. Sun. Delving deep into rectifiers:
Surpassing human-level performance on imagenet classification. In
ICCV, 2015.
"""

class ResBlock(Model):
  def __init__(self, channels, down_sample=False):
    super().__init__()
    self.__channels = channels
    self.__down_sample = down_sample
    self.__strides = [2, 1] if down_sample else [1, 1]

    self.conv1 = Conv2D(self.__channels, strides=self.__strides[0], kernel_size=(3, 3), padding="same", kernel_initializer="he_normal")
    self.bn1 = BatchNormalization()
    self.conv2 = Conv2D(self.__channels * 2, strides=self.__strides[1], kernel_size=(3, 3), padding="same", kernel_initializer="he_normal")
    self.bn2 = BatchNormalization()
    self.merge = Add()
    if self.__down_sample:
      self.res_conv = Conv2D(self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer="he_normal", padding="same")
      self.res_bn = BatchNormalization()

  def call(self, inputs):
    res = inputs
    x = self.conv1(inputs)
    x = self.bn1(x)
    x = tf.nn.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    if self.__down_sample:
      res = self.res_conv(res)
      res = self.res_bn(res)
    x = self.merge([x, res])
    out = tf.nn.relu(x)
    return out


class ResNet(Model):
  def __init__(self, **kwargs):
    """num_classes: number of classes in specific classification task."""
    super().__init__(**kwargs)
    self.conv1 = Conv2D(64, (7, 7), strides=2, padding="same", kernel_initializer="he_normal")
    self.bn = BatchNormalization()

    self.maxpool = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")

    self.res1_1 = ResBlock(64)
    self.res1_2 = ResBlock(64)

    self.res2_1 = ResBlock(128, down_sample=True)
    self.res2_2 = ResBlock(128)

    self.res3_1 = ResBlock(256, down_sample=True)
    self.res3_2 = ResBlock(256)

    self.res4_1 = ResBlock(512, down_sample=True)
    self.res4_2 = ResBlock(512)

    self.avgpool = GlobalAveragePooling2D()

    self.fc1 = Dense(1, activation="relu")

  def call(self, inputs):
    out = self.conv1(inputs)
    out = self.bn(out)
    out = tf.nn.relu(out)
    out = self.maxpool(out)
    for res_block in [self.res1_1, self.res1_2, self.res2_1, self.res2_2, self.res3_1, self.res3_2, self.res4_1, self.res4_2]:
      out = res_block(out)
    out = self.avgpool(out)
    out = Flatten(out)
    out = self.fc1(out)
    return out

with strategy.scope():
  model = ResNet()
  model.build(input_shape = (None, 125, 125, 4))
  model.compile(optimizer=optimiseur, steps_per_execution=exec_step, loss=criterion, metrics=["mse"])
"""
model = ResNet()
tpu_model = tf.compat.v1.estimator.tpu.keras_to_tpu_model(
    model,
    strategy=strategy)
with strategy.scope():
  tpu_model.build(input_shape = (None, 125, 125, 4))
  tpu_model.compile(optimizer=optimiseur, steps_per_execution=exec_step, loss=criterion, metrics=["mse"])
"""
model.summary()

In [ ]:
# @title tensor loop
# print(f"E [{epoch + 1}/{epochs}], L: {running_loss:.4f}, T: [{datetime.datetime.now()}]")
#steps_per_epoch=steps_per_epoch
batch_size_parq=50
print("Training started at", datetime.datetime.now())
plotloss = []
for epoch in range(epochs):
  for parq in parqs:
    parq_file = pq.ParquetFile(folder+parqs[0])
    for i in parq_file.iter_batches(batch_size=batch_size_parq, columns=['X_jet', 'm']):
      df = i.to_pandas()
      X = df['X_jet'].values
      y = df['m'].values
      X = make_Xs(X)
      print(X.shape)
      history = model.fit(x=X, y=y, batch_size=batch_size_train, epochs=1, steps_per_epoch=steps_per_epoch, shuffle=True)
      plotloss.append(sqrt(history.history['mse']))
      print(datetime.datetime.now())
print("Training completed at", datetime.datetime.now())
print(f"Test RMSE: [{sqrt(model.evaluate(x=X_test, y=Y_test, batch_size=batch_size))}], T: [{datetime.datetime.now()}]")